#  Submit Phonons Calculation

In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.code import Code
from aiida.orm import load_node
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.base import Int, Float, Str, Bool
from aiida.work.run import submit
from aiida.orm.data.structure import StructureData

import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import nglview

from apps.surfaces.structure_browser import StructureBrowser
from apps.surfaces.widgets.computer_code_selection import ComputerCodeDropdown
from apps.surfaces.widgets.dft_details_dev import DFTDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.suggested_param import suggested_parameters

#from apps.surfaces.widgets.submit_leo import SubmitWorkChainWidget

# Phonon specific import

from apps.phonons.widgets.phonons_details import PHNSDetails
from apps.phonons.widgets.slab_validity import slab_is_valid
from apps.phonons.widgets.submit_button_dev import SubmitButton
from apps.phonons.phonons_work import PhononsWorkchain


from ase.data import vdw_radii

from apps.surfaces.widgets import find_mol


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
## GENERAL PARAMETERS
the_workchain=PhononsWorkchain
job_details = {'workchain':'PhononsWorkchain'}
job_details['calc_type']='Full DFT'
job_details['nreplicas']=1
job_details['proc_node']=12
job_details['nproc_rep']=job_details['nreplicas']*job_details['proc_node']
job_details['ncalcs']=6000
job_details['num_cores_per_mpiproc']=1

In [ ]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    slab_analyzed = find_mol.analyze_slab(atoms)
    job_details['slab_analyzed'] = slab_analyzed
    job_details['elements']      = slab_analyzed['all_elements']
    cell=job_details['slab_analyzed']['the_cell']
    cell=str(cell[0][0])+' '+str(cell[1][1])+' '+str(cell[2][2])    
    dft_details_widget.reset(cell=cell)
    
    #guess_calc_params(slab_analyzed)
    job_details['ncalcs']=6*len(atoms)
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
#def guess_calc_params(slab_analyzed):
#    method = dft_details_widget.calc_type.value
#    valid_slab, msg = slab_is_valid(slab_analyzed,method)
#    if valid_slab:        
#        atoms_to_fix,num_nodes=suggested_parameters(slab_analyzed,method)
#        dft_details_widget.reset(fixed_atoms=atoms_to_fix,num_machines=num_nodes,calc_type=method)
#    else:
#        print(msg)
def update_nproc_rep(c):
    num_nodes = dft_details_widget.num_machines.value
    proc_node = phns_details_widget.proc_node.value
    phns_details_widget.proc_rep.value=num_nodes*proc_node
    
def get_nfixed(slab_analyzed):
    
    method = dft_details_widget.calc_type.value
    valid_slab, msg = slab_is_valid(slab_analyzed,method)
    if method in ['Mixed DFTB', 'Mixed DFT']:
        full_slab=slab_analyzed['slabatoms'] + slab_analyzed['bottom_H']
        full_slab = [i for i in full_slab] 
        atoms_to_fix = find_mol.mol_ids_range(full_slab)
        nfixed = len(atoms_to_fix)

    if method in ['Full DFT']:
        partial_slab = slab_analyzed['bottom_H'] + slab_analyzed['slab_layers'][0] + slab_analyzed['slab_layers'][1]
        partial_slab = [i for i in partial_slab] 
        atoms_to_fix = find_mol.mol_ids_range(partial_slab)
        nfixed = len(atoms_to_fix)
    
            
    return nfixed
            
        
# dealt with by submit button    
#def valid_config():
#    num_calcs = phns_details_widget.num_calcs.value
#    num_rep = phns_details_widget.num_rep.value
#    if not np.mod(num_calcs, num_rep)==0 :
#        print('Choose #Replicas so that #Calculations % #Replicas is 0.')
    

In [ ]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##STRUCTURE
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value') 

##VIEWER
viewer_widget = ViewerDetails()
mol_ids_info_out = ipw.Output()
display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

##CODE
computer_code_dropdown = ComputerCodeDropdown(job_details=job_details,
                                              input_plugin='cp2k'
                                             )

##DFT
dft_details_widget = DFTDetails(job_details           = job_details,
                                widgets_disabled      = {
                                    'calc_type'     : True,
                                    'max_force'     : True
                                }
                               )    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
#dft_details_widget.calc_type.observe(guess_calc_params, 
#                                     names='value'
#                                    )
dft_details_widget.num_machines.observe(update_nproc_rep,
                                        names='value'
                                       )
##PHONON
phns_details_widget = PHNSDetails(job_details=job_details) 
phns_details_widget.proc_node.observe(update_nproc_rep,
                                        names='value'
                                     )

# dealt with by submit button
#phns_details_widget.num_rep.observe(lambda c: valid_config(),
#                                        names='value'
#                                       )
#phns_details_widget.num_calcs.observe(lambda c: valid_config(),
#                                        names='value'
#                                       )

#display code dropdown
display(ipw.VBox([computer_code_dropdown, dft_details_widget, 
                  phns_details_widget]))

#display submit button
btn_submit = SubmitButton(
    the_workchain=the_workchain,job_details=job_details,
    presub_calls=[dft_details_widget.read_widgets_and_update_job_details]
)
display(btn_submit)

#widget = SubmitWorkChainWidget(workchain=MoleculeOptWorkChain, validate_fn=validate_cp2k)
#display(widget)